
HERE API:

* Route trace: return=polyline (visible area of 100m left and right)

* Route duration and length: return=travelSummary

* Route traffic: ratio between the no-traffic route duration and the route duration considering traffic / corridor using route flexible polyline (only steps)


OSM FILE(?): 

* Route nature: osm nature polygon intersection with visible area

* Route attractions: number of attractions inside route visible area


OPEN DATA:

* Route crimes: sum of severity of crimes inside visible area

* Route accidents: sum of severity of accidents inside visible area



1 - Get routes trace, duration and length of every intermodal

2 - Get traffic data

3 - Associate route trace (visible area) with nature,attractions,crime and accidents

In [21]:
from env import env
import flexpolyline as fp
import urllib.request
import json, sys
import pandas as pd


In [11]:
polyline = json.load(open('results/dumps/polyline.json'))
traffic = json.load(open('results/dumps/travelSummary.json'))

In [30]:
sections = traffic['routes'][0]['sections']


for section in sections:
    print(f"section {section['id']}:\n{section}\n")
    

section R0-S0:
{'id': 'R0-S0', 'type': 'pedestrian', 'travelSummary': {'duration': 15, 'length': 14}, 'departure': {'time': '2023-09-04T15:13:00+02:00', 'place': {'type': 'place', 'location': {'lat': 52.531056, 'lng': 13.384945}}}, 'arrival': {'time': '2023-09-04T15:13:15+02:00', 'place': {'type': 'place', 'location': {'lat': 52.531022, 'lng': 13.385153}}}, 'transport': {'mode': 'pedestrian'}}

section R0-S1:
{'id': 'R0-S1', 'type': 'rented', 'preActions': [{'action': 'setup', 'duration': 60}], 'postActions': [{'action': 'park', 'duration': 60}], 'travelSummary': {'duration': 917, 'length': 3517}, 'departure': {'time': '2023-09-04T15:14:15+02:00', 'place': {'type': 'place', 'location': {'lat': 52.531022, 'lng': 13.385153}}}, 'arrival': {'time': '2023-09-04T15:29:32+02:00', 'place': {'type': 'place', 'location': {'lat': 52.508911, 'lng': 13.368039}}}, 'transport': {'mode': 'bicycle'}, 'agency': {'id': 'nextbike_bn', 'name': 'nextbike Berlin', 'website': 'https://www.nextbike.de/de/berli

In [2]:
apiKey = env('HERE_API_KEY')
altNum = 1
origin = '52.53105637575095,13.384944833815183'
destination = '52.40358749909618,13.058351363288239'

returnType = {
    0:'polyline',
    1:'travelSummary' # try: summary,polyline (!use summary,polyline!)
}

vehicle = { # for non rented/shared vehicles
    0:'car',
    1:'bicycle',
    2:''
}

transit = {
    0:'bus',
    1:''
}

routeArea = {
    0:'routeTail',
    1:'routeHead',
    2:'entireRoute'
}

taxi = {0:'car'} # for taxi
rented = {0:'bicycle'} # for bike share

url = f'https://intermodal.router.hereapi.com/v8/routes?apiKey={apiKey}&alternatives={altNum}&destination={destination}&origin={origin}'

# taxi
mode = f''

# bus
mode = f''

# bike share
mode = f''

# taxi / bike share
#mode = f'&vehicle[modes]={vehicle[0]}&transit[modes]={transit[0]}&vehicle[enable]={routeArea[0]}&vehicle[enable]={routeArea[1]}'
mode = f'&taxi[modes]=car&rented[modes]=bicycle&taxi[enable]=routeHead&rented[enable]=routeTail&transit[enable]=' #

# taxi / bus
#mode = f'&vehicle[modes]={vehicle[0]}&transit[modes]={transit[0]}&vehicle[enable]={routeArea[0]}&vehicle[enable]={routeArea[1]}'
mode = f'&taxi[modes]=car&transit[modes]=bus&taxi[enable]=routeHead&transit[enable]=routeTail' #

# bike share / taxi
#mode = f'&vehicle[modes]={vehicle[0]}&transit[modes]={transit[0]}&vehicle[enable]={routeArea[0]}&vehicle[enable]={routeArea[1]}' 
mode = f'rented[modes]=bicycle&taxi[modes]=car&rented[enable]=routeHead&taxi[enable]=routeTail&transit[enable]=' #

# bike share / bus
#mode = f'&vehicle[modes]={vehicle[0]}&transit[modes]={transit[0]}&vehicle[enable]={routeArea[0]}&vehicle[enable]={routeArea[1]}'
mode = f'' #

# bus / taxi
#mode = f'&vehicle[modes]={vehicle[0]}&transit[modes]={transit[0]}&vehicle[enable]={routeArea[0]}&vehicle[enable]={routeArea[1]}'
mode = f'' #

# bus / bike share
#mode = f'&vehicle[modes]={vehicle[0]}&transit[modes]={transit[0]}&vehicle[enable]={routeArea[0]}&vehicle[enable]={routeArea[1]}'
mode = f'' #

request = url + mode + f'&return={returnType[0]}' 

response_poly = urllib.request.urlopen(request).read().decode('utf-8')
response_poly = json.loads(response_poly)
json.dump(response_poly, open('results/dumps/polyline.json', 'w+'),indent=4)

request = url + mode + f'&return={returnType[1]}' 

response_summ = urllib.request.urlopen(request).read().decode('utf-8') 
response_summ = json.loads(response_summ)
json.dump(response_summ, open('results/dumps/travelSummary.json', 'w+'),indent=4)

In [15]:
print(response_poly['routes'][0]['sections'][1]['polyline'])

False


In [18]:
import googlemaps
import gmplot

originCoord = origin.split(',')

output_map = gmplot.GoogleMapPlotter(originCoord[0],originCoord[1], 12)

colors = {
    'pedestrian':'blue',
    'bus':'yellow',
    'bicycle':'green'
}

first = True
for i in response_poly['routes'][0]['sections']:
    
    lat = []
    lng = []
    
    if first:
        lat.append(i['departure']['place']['location']['lat'])
        lng.append(i['departure']['place']['location']['lng'])
        
    first = False
    
    try:
        polyline = fp.decode(i['polyline'])
        
        for coord in polyline:
            lat.append(coord[0])

            
            lng.append(coord[1])
    except:
        pass
        
    lat.append(i['arrival']['place']['location']['lat'])
    lng.append(i['arrival']['place']['location']['lng'])

    output_map.plot(lat, lng, colors[i['transport']['mode']], edge_width=3)

output_map.draw('results/map_plot/Here_map.html')

# traffic api
#https://data.traffic.hereapi.com/v7/flow?locationReferencing=shape&in=bbox:13.400,52.500,13.405,52.505" example

https://intermodal.router.hereapi.com/v8/routes?apiKey=RhbiMBCRVNDFplxIdn-yII6eZU9upCqvuChA2n7aSXo&alternatives=1&destination=52.40358749909618,13.058351363288239&origin=52.53105637575095,13.384944833815183&return=polyline

for traffic data:

https://developer.here.com/documentation/traffic-api/dev_guide/topics/concepts/flow.html#flow

https://developer.here.com/documentation/traffic-api/dev_guide/topics/concepts/geospatial-filters.html